In [ ]:
from Notebook.DSNotebook.NotebookExecutor import NotebookExecutor
nb = NotebookExecutor()
data = nb.get_data('59801689926743119', '@SYS.USERID', 'True', {}, [])
data.head(22)

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
data.select_dtypes('object').columns

In [ ]:
data.select_dtypes('number').columns

In [ ]:
#Preprocessing

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

def preprocess_categorical(df_in, categorical_columns):
    df = df_in[categorical_columns].copy()
    ohe = OneHotEncoder()
    df_cat = ohe.fit_transform(df);
    df_cat = df_cat.todense()
    return df_cat, ohe


def preprocess_numerical(df_in, numerical_columns):
    df = df_in[numerical_columns].copy()
    scaler = MinMaxScaler()
    df_num = scaler.fit_transform(df);
    return df_num, scaler


def preprocess_data(df_in, categorical_columns, numerical_columns):
    df_cat, ohe = preprocess_categorical(df_in, categorical_columns)
    df_num, scaler = preprocess_numerical(df_in, numerical_columns)
# RAISE ISSUE: WHEN CELL IS PRINTING SOMETHING, ERROR MESSAGE IS TRUNCATED
    X = np.concatenate((df_cat, df_num), axis=1)
    return X, ohe, scaler



categorical_columns = ['gender', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod']

numerical_columns = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']

X, ohe, scaler = preprocess_data(data, categorical_columns, numerical_columns)
y = data['Churn']

In [ ]:
# Modelling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = RandomForestClassifier()
model.fit(X_train, y_train);

preds_train = model.predict(X_train)
preds_test = model.predict(X_test)

print(classification_report(y_train, preds_train))
print(classification_report(y_test, preds_test))

In [ ]:
# make column transformer object to use for data preprocessing in pipeline

from sklearn.compose import ColumnTransformer

categorical_columns = ['gender', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod']

numerical_columns = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']

ct = ColumnTransformer(
    [
        ('ohe', OneHotEncoder(), categorical_columns),
        ('scaler', MinMaxScaler(), numerical_columns)
    ],
    remainder='drop'
)

X_trans = ct.fit_transform(data);